# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir_assg3_eithan' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        paths.append(full_path+b.name)

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

<H1> Helper function to create Indexes <H1/>

In [ ]:
index_type = "_anchor"

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    cnt = Counter([tok for tok in tokens if tok not in all_stopwords])
    return [(key, (id, val)) for key,val in cnt.items()]
    
    
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key= lambda tup: tup[0])


def calculate_df(postings):
    return postings.map(lambda lm: (lm[0],len(lm[1])))


def partition_postings_and_write(postings):
    partition_rdd = postings.map(lambda w_pl: (token2bucket_id(w_pl[0]),w_pl)).groupByKey()
    return partition_rdd.map(lambda b_w_pl: InvertedIndex.write_a_posting_list(b_w_pl,bucket_name,index_type=index_type))

def word_count_binary(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    words_set = set(tok for tok in tokens if tok not in all_stopwords)
    return [(word, (id, 1)) for word in words_set]

def doc_len_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    return (id,len(tokens))

# Assignment2 Code: 

def tokenize(text):
  return [token.group() for token in RE_WORD.finditer(text.lower())]

def calculate_tf(docs):
  word_counts = Counter()
  for text in docs:
    tokens = tokenize(title)
    doc_counts = Counter(tokens)
    word_counts.update(doc_counts)
  return word_counts

<H1> Creating Anchor Text Small Inverted Index <H1/>

In [ ]:

def parse_anchor_text(pair):
    id = pair[1]
    anchor_text = ""
    for tup in pair[0]:
        anchor_text += " " + tup[1]
    return (anchor_text, id)

tmp_pairs = doc_anchor_pairs.map(lambda pair: parse_anchor_text(pair))


In [ ]:

# word counts map
word_counts_anchor = tmp_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
w2df_anchor = calculate_df(postings_anchor)
w2df_anchor_dict = w2df_anchor.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=f'postings_gcp{index_type}'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
# Create inverted index instance
anchor_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
anchor_index.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
anchor_index.df = w2df_anchor_dict
# write the global stats out
anchor_index.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
